# Setup and Import Libraries

In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

# Embeddings

## OPENAI Embeddings

In [4]:
embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000023AFD029950>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000023ADDF80390>, model='text-embedding-3-large', dimensions=3072, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:
text = "Understanding OpenAI Embeddings"

In [6]:
query_result = embeddings.embed_query(text=text)
query_result

[0.005841143894940615,
 0.03672826290130615,
 -0.01822661980986595,
 -0.03860342875123024,
 0.004841055255383253,
 -0.002572103636339307,
 0.01602642424404621,
 0.02625233307480812,
 0.0032002844382077456,
 -0.022714518010616302,
 -0.01075095497071743,
 0.007775690872222185,
 -0.0037753356155008078,
 0.0022908286191523075,
 -0.012507361359894276,
 0.01551387831568718,
 0.02662736549973488,
 0.051404569298028946,
 -0.022927038371562958,
 -0.023939628154039383,
 -0.016951506957411766,
 -0.007650679908692837,
 -0.04122866317629814,
 -0.05735509842634201,
 0.025977307930588722,
 -0.03355298191308975,
 0.005087952129542828,
 0.02862754464149475,
 0.005491112824529409,
 0.05920526012778282,
 0.023202061653137207,
 -0.03452806919813156,
 0.01163228414952755,
 -0.010994726791977882,
 -0.02476469986140728,
 0.016626477241516113,
 0.042503777891397476,
 0.038553427904844284,
 -0.03885345160961151,
 0.031502798199653625,
 0.01761406473815441,
 0.003056521527469158,
 0.011219747364521027,
 0.00365

len(query_result)

In [7]:
query_result[0]

0.005841143894940615

### Text Splitting and OpenAI Embeddings

In [8]:
loader = TextLoader('speech.txt')
docs = loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_

### Vector Database

In [10]:
db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)
db

### Query Vector Database

In [11]:
query = "It will be all the easier for us to conduct ourselves as belligerents"

In [12]:
retrieved_results = db.similarity_search(query=query)
retrieved_results

[Document(metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'s

In [13]:
retrieved_results[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early'

## Ollama

Ollama supports embedding models, making it possible to build Retrieval Augmented Generation (RAG) applications that combine text prompts with existing documents or other data

embeddings = (
    OllamaEmbeddings(model='gemma:2b')  ## By Default = llama2
)

In [14]:
r1 = embeddings.embed_documents(
    [
        "Alpha is the first letter of Greek Alphabet",
        "Beta is the second letter of Greek Alphabet"
    ]
)

r1

[[-0.002833691891282797,
  -0.02710052952170372,
  0.0012444303138181567,
  0.00824736524373293,
  -0.012957083061337471,
  -0.03873546048998833,
  0.027829498052597046,
  0.014979617670178413,
  0.020153874531388283,
  -0.04039350897073746,
  -0.014200620353221893,
  0.02174045518040657,
  -0.0243704654276371,
  0.0008102643769234419,
  -0.002772944513708353,
  0.003001640783622861,
  -0.0008026708965189755,
  0.018896043300628662,
  -0.00027783046243712306,
  0.008840546943247318,
  -0.03550512343645096,
  0.008168751373887062,
  -0.02915879711508751,
  -0.0028247584123164415,
  -0.005656663794070482,
  -0.005817465949803591,
  0.002444193232804537,
  -0.010612944141030312,
  0.008861986920237541,
  -0.0027389973402023315,
  0.042451776564121246,
  -0.014229207299649715,
  0.04233742877840996,
  0.012178085744380951,
  -0.05285746604204178,
  -0.012785560451447964,
  -0.0131571926176548,
  0.038849808275699615,
  -0.004481020849198103,
  0.019567839801311493,
  0.020739909261465073,


In [15]:
len(r1[0])

3072

In [16]:
# r1[0]

In [17]:
query = "What is the second letter of Greek Alphabet"

embeddings.embed_query(text=query)

[0.010901201516389847,
 -0.03463602066040039,
 0.004735162947326899,
 0.043103259056806564,
 0.015208576805889606,
 -0.02959108166396618,
 0.05283911153674126,
 0.011535506695508957,
 -0.005952144507318735,
 -0.030446656048297882,
 -0.009344940073788166,
 -0.008644253946840763,
 -0.033573929220438004,
 0.018335849046707153,
 0.025209950283169746,
 0.014205489307641983,
 -0.012184563092887402,
 0.026950601488351822,
 0.006615952122956514,
 0.031213723123073578,
 -0.0388253889977932,
 0.012981132604181767,
 0.0203420240432024,
 -0.0101710120216012,
 0.0011284734355285764,
 -0.001538743614219129,
 0.017568781971931458,
 -0.005362092982977629,
 0.02287924662232399,
 0.030328646302223206,
 0.011506004258990288,
 0.01332041248679161,
 0.054196227341890335,
 -0.008585249073803425,
 -0.03531458228826523,
 -0.019501201808452606,
 -0.010082504712045193,
 0.01443413458764553,
 -0.02130085788667202,
 -0.01649193838238716,
 0.024929674342274666,
 0.0029502573888748884,
 -0.052603088319301605,
 0.01

## HuggingFace Embeddings

In [18]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
text = "This is a text document"

In [20]:
query_result = embeddings.embed_query(text=text)

In [21]:
len(query_result)

384

In [22]:
# query_result